In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import metrics  
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, AdamW

In [1]:
# Step 1: Load the data
# Load embeddings and labels
embeddings_1 = np.load('embeddings_1.npy')
embeddings_2 = np.load('embeddings_2.npy')
embeddings = np.vstack([embeddings_1, embeddings_2])  # Combine both embedding files

NameError: name 'np' is not defined

In [3]:
# Load labels and convert them to multi-hot encoding
with open('icd_codes_1.txt') as f1, open('icd_codes_2.txt') as f2:
    labels_1 = [line.strip().split(';') for line in f1]
    labels_2 = [line.strip().split(';') for line in f2]
    labels = labels_1 + labels_2

In [4]:
# Create a mapping for ICD10 codes to multi-hot encoding
unique_codes = sorted(set(code for sublist in labels for code in sublist))
code_to_index = {code: idx for idx, code in enumerate(unique_codes)}
num_classes = len(unique_codes)

In [5]:
# Convert labels to multi-hot vectors
y = np.zeros((len(labels), num_classes), dtype=int)
for i, label_list in enumerate(labels):
    for code in label_list:
        y[i, code_to_index[code]] = 1

In [6]:
# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(embeddings, y, test_size=0.2, random_state=42)

In [3]:
# Hyperparameters
embedding_dim = 1024
num_classes = 1400
filter_sizes = [1, 2, 3, 4]
num_filters = 128
dropout_rate = 0.5

# Model Architecture
input_layer = Input(shape=(embedding_dim, 1))

# Convolution and Pooling layers
conv_layers = []
for filter_size in filter_sizes:
    conv = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(input_layer)
    pool = GlobalMaxPooling1D()(conv)
    conv_layers.append(pool)

# Concatenate all pooled features
concat = Concatenate()(conv_layers)

# Dropout layer
dropout = Dropout(dropout_rate)(concat)

# Output layer with sigmoid activation for multi-label classification
output_layer = Dense(num_classes, activation='sigmoid')(dropout)

# Model compilation
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=[AUC(multi_label=True, name="micro_auc")])




In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1024, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1024, 128) │        256 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1023, 128) │        384 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1022, 128) │        512 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 1021, 128) │        640 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1400)      │    718,200 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 719,992 (2.75 MB)

 Trainable params: 719,992 (2.75 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Training the model
X_train_expanded = np.expand_dims(X_train, axis=-1)  # Expanding dimensions for Conv1D
early_stopping = EarlyStopping(monitor='val_micro_auc', mode='max', patience=5, restore_best_weights=True)

In [14]:
history = model.fit(X_train_expanded, y_train,
                    validation_split=0.1,
                    epochs=2,
                    batch_size=128,
                    callbacks=[early_stopping])

Epoch 1/2
  75/1120 ━━━━━━━━━━━━━━━━━━━━ 1:48 103ms/step - loss: 0.6274 - micro_auc: 0.3314

KeyboardInterrupt: 

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Input
# from tensorflow.keras.optimizers import Adam

# # Function to build the model with specific parameters
# def build_model(learning_rate=0.0005):
#     model = Sequential([
#         Input(shape=(1024,)),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(256, activation='relu'),
#         Dropout(0.3),
#         Dense(128, activation='relu'),
#         Dropout(0.3),
#         Dense(64, activation='relu'),
#         Dropout(0.3),
#         Dense(1400, activation='sigmoid')
#     ])
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
#     return model


In [ ]:
# import numpy as np

# # Number of models for bagging
# num_models = 10
# models = []

# # Train each model on a bootstrap sample of the data
# for i in range(num_models):
#     print(f"Training model {i + 1}/{num_models}")
    
#     # Create a bootstrap sample (randomly sample with replacement)
#     indices = np.random.choice(len(X_train), len(X_train), replace=True)
#     X_bootstrap, y_bootstrap = X_train[indices], y_train[indices]
    
#     # Build and train a new model on the bootstrap sample
#     model = build_model(learning_rate=0.0005)
#     model.fit(X_bootstrap, y_bootstrap, epochs=2, batch_size=128, validation_data=(X_val, y_val), verbose=0)
    
#     # Append trained model to the list
#     models.append(model)


Training model 1/10
Training model 2/10
Training model 3/10
Training model 4/10
Training model 5/10
Training model 6/10
Training model 7/10
Training model 8/10
Training model 9/10
Training model 10/10


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Input
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
# from tensorflow.keras.optimizers.schedules import ExponentialDecay

# def build_model(learning_rate):
#     model = Sequential([
#         Input(shape=(1024,)),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(256, activation='relu'),
#         Dropout(0.3),
#         Dense(128, activation='relu'),
#         Dropout(0.3),
#         Dense(1400, activation='sigmoid')
#     ])
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    # return model


In [ ]:
# # Grid of learning rate options and schedulers
# param_grid = {
#     "learning_rate": [0.001, 0.0005, 0.0001],
#     "scheduler": [
#         None,  # No scheduler
#         ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9),  # Exponential Decay
#         ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001),  # ReduceLROnPlateau
#         LearningRateScheduler(lambda epoch: 0.001 * (0.5 ** (epoch // 5)))  # Step Decay
#     ]
# }


In [ ]:
# from sklearn.model_selection import ParameterGrid
# from tensorflow.keras.optimizers.schedules import ExponentialDecay

# # Initialize best tracking variables
# best_val_loss = float("inf")
# best_params = None
# results = []

# # Iterate over each combination in the parameter grid
# for params in ParameterGrid(param_grid):
#     print(f"Testing params: {params}")
    
#     # Handle learning rate schedules
#     if isinstance(params["scheduler"], ExponentialDecay):
#         # Use ExponentialDecay as the learning rate in the optimizer
#         optimizer = Adam(learning_rate=params["scheduler"])
#         callbacks = []  # No callbacks needed for ExponentialDecay
#     else:
#         # Set a fixed learning rate and add ReduceLROnPlateau or LearningRateScheduler as callbacks
#         optimizer = Adam(learning_rate=params["learning_rate"])
#         callbacks = [params["scheduler"]] if params["scheduler"] is not None else []

#     # Build the model with the specified optimizer
#     model = Sequential([
#         Input(shape=(1024,)),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(512, activation='relu'),
#         Dropout(0.3),
#         Dense(256, activation='relu'),
#         Dropout(0.3),
#         Dense(128, activation='relu'),
#         Dropout(0.3),
#         Dense(1400, activation='sigmoid')
#     ])
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

#     # Train the model
#     history = model.fit(X_train, y_train, epochs=10, batch_size=128, 
#                         validation_data=(X_val, y_val), callbacks=callbacks, verbose=0)

#     # Evaluate the model on the validation set
#     val_loss = history.history['val_loss'][-1]
#     results.append((params, val_loss))

#     # Track the best model based on validation loss
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_params = params

# # Display the best parameters and validation loss
# print("Best Params:", best_params)
# print("Best Validation Loss:", best_val_loss)


Testing params: {'learning_rate': 0.001, 'scheduler': None}
Testing params: {'learning_rate': 0.001, 'scheduler': <keras.src.optimizers.schedules.learning_rate_schedule.ExponentialDecay object at 0x0000025FB5D889D0>}
Testing params: {'learning_rate': 0.001, 'scheduler': <keras.src.callbacks.reduce_lr_on_plateau.ReduceLROnPlateau object at 0x0000025FB5D885B0>}
Testing params: {'learning_rate': 0.001, 'scheduler': <keras.src.callbacks.learning_rate_scheduler.LearningRateScheduler object at 0x0000025FB5D888E0>}
Testing params: {'learning_rate': 0.0005, 'scheduler': None}
Testing params: {'learning_rate': 0.0005, 'scheduler': <keras.src.optimizers.schedules.learning_rate_schedule.ExponentialDecay object at 0x0000025FB5D889D0>}
Testing params: {'learning_rate': 0.0005, 'scheduler': <keras.src.callbacks.reduce_lr_on_plateau.ReduceLROnPlateau object at 0x0000025FB5D885B0>}
Testing params: {'learning_rate': 0.0005, 'scheduler': <keras.src.callbacks.learning_rate_scheduler.LearningRateScheduler

In [5]:
from tensorflow.keras.optimizers import Adam

# Build the model with the optimal parameters
model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1400, activation='sigmoid')
])

# Compile the model with the best learning rate found
model.compile(optimizer=AdamW(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy',metrics.Precision(),metrics.Recall()])
model.summary()

c:\Users\himan\.conda\envs\hj\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1400)           │       180,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 869,624 (3.32 MB)

 Trainable params: 869,624 (3.32 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the model with the optimal parameters
history = model.fit(X_train, y_train, epochs=2, batch_size=128, validation_data=(X_val, y_val))

# Check the performance on the validation set
val_loss = history.history['val_loss'][-1]
print("Final Validation Loss:", val_loss)# 

Epoch 1/2
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5674 - loss: 0.0021 - precision_1: 0.8518 - recall_1: 0.6615 - val_accuracy: 0.5649 - val_loss: 0.0018 - val_precision_1: 0.8661 - val_recall_1: 0.7267
Epoch 2/2
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.5646 - loss: 0.0021 - precision_1: 0.8536 - recall_1: 0.6606 - val_accuracy: 0.5771 - val_loss: 0.0018 - val_precision_1: 0.8672 - val_recall_1: 0.7312
Final Validation Loss: 0.0017785148229449987


In [ ]:
# # Step 2: Define the model
# model = Sequential([
#     Dense(512, activation='relu', input_shape=(1024,)),
#     Dropout(0.4),
#     Dense(256, activation='relu'),
#     Dropout(0.4),
#     Dense(128, activation='relu'),
#     Dropout(0.4),
#     Dense(num_classes, activation='sigmoid')  # Sigmoid for multi-label classification
# ])

# model.compile(optimizer=RMSprop(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy',metrics.Precision(),metrics.Recall()])

In [ ]:
# # Step 3: Train the model
# model.fit(X_train, y_train, epochs=3, batch_size=128, validation_data=(X_val, y_val))

Epoch 1/3
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.0830 - loss: 0.0188 - precision_4: 0.0814 - recall_4: 0.0764 - val_accuracy: 0.3532 - val_loss: 0.0049 - val_precision_4: 0.6668 - val_recall_4: 0.3086
Epoch 2/3
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.2943 - loss: 0.0055 - precision_4: 0.6830 - recall_4: 0.2415 - val_accuracy: 0.4202 - val_loss: 0.0038 - val_precision_4: 0.7227 - val_recall_4: 0.4406
Epoch 3/3
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.3672 - loss: 0.0046 - precision_4: 0.7199 - recall_4: 0.3129 - val_accuracy: 0.4596 - val_loss: 0.0032 - val_precision_4: 0.8054 - val_recall_4: 0.4581


In [24]:
# Step 1: Load the test embeddings
test_embeddings = np.load('test_data.npy')

In [ ]:
# # Placeholder to store predictions from each model
# predictions = []

# # Collect predictions from each model in the ensemble
# for model in models:
#     pred = model.predict(test_embeddings)
#     predictions.append(pred)

# # Average the predictions from all models
# avg_predictions = np.mean(predictions, axis=0)

3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step


In [ ]:
# # Placeholder to store predictions from each model
# predictions = []

# # Collect predictions from each model in the ensemble
# for model in models:
#     pred = model.predict(X_val)
#     predictions.append(pred)

# # Average the predictions from all models
# avg_predictions = np.mean(predictions, axis=0)

1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1244/1244 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [ ]:
# # Apply a threshold to convert probabilities into binary labels
# # (for example, threshold = 0.5 or another value tuned for your needs)
# threshold = 0.26
# final_predictions = (avg_predictions >= threshold).astype(int)

In [25]:
# Step 2: Make predictions on the test data
# Load the model (assuming you've already trained and saved it if needed)
y_test_pred = model.predict(test_embeddings)

3110/3110 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [26]:
for i in range(10000):
    if y_test_pred[i][938]>=0.4:
        print('yes')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [31]:
# Step 4: Evaluate the model
# Predict on the validation set and compute micro F2 score
y_val_pred = model.predict(X_val) > 0.41 # Convert probabilities to binary predictions
micro_f2_score = f1_score(y_val, y_val_pred, average='micro')

print(f'Micro F2 Score on validation set: {micro_f2_score:.4f}')


1244/1244 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Micro F2 Score on validation set: 0.7973


In [60]:
# Step 3: Convert predictions to ICD10 code labels
# Apply threshold to determine which codes to include (e.g., 0.5 threshold)
threshold = 0.525
test_labels_pred = (y_test_pred > threshold).astype(int)

In [61]:
# Step 4: Create a submission file in the specified format
# Map indices back to ICD10 codes
index_to_code = {v: k for k, v in code_to_index.items()}

submission_data = []
for idx, label_vector in enumerate(test_labels_pred, start=1):
    # Get codes with predictions above the threshold and sort lexicographically
    codes = [index_to_code[i] for i, val in enumerate(label_vector) if val == 1]
    codes = sorted(codes)  # Sort lexicographically
    label_string = ';'.join(codes) if codes else ''  # Stitch with ';' or leave blank if no label
    submission_data.append({'id': idx, 'labels': label_string})

In [62]:
# Convert to DataFrame and save as CSV
submission_df = pd.DataFrame(submission_data)
submission_df

,id,labels
0,1,G56.21
1,2,S83.242A
2,3,G56.01
3,4,M65.312
4,5,S83.241A;S83.281A
...,...,...
99485,99486,D12.0;K57.30;K63.5
99486,99487,K29.50
99487,99488,D12.2;D12.5;K64.8;Z12.11
99488,99489,B96.81;K21.9;K29.50


In [63]:
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

Submission file 'submission.csv' created successfully.
